### NORMALIZE CREATING NEW TABLE

ta metoda polega na tworzeniu tabeli dla każdej listy

In [2]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

In [3]:
paths_test = [ '../data-test/airlines.json', 
            '../data-test/gists.json', 
            '../data-test/historical-events.json',#have to rename the same value in json (movie on  the same level)
            '../data-test/movies.json',
            '../data-test/reddit.json',#bad
            '../data-test/nasa.json'
            ]

In [4]:
paths = [ '../data/airlines.json', 
            '../data/gists.json', 
            '../data/historical-events.json',
            '../data/movies.json',
            '../data/reddit.json',
            '../data/nasa.json']

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                flatten(a, attribute_name)
                i += 1
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out

results = []

for path in paths_test:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)
        
    if isinstance(dt, list):
        flattened_data = [flatten_json(item) for item in dt]
    else:
        flattened_data = [flatten_json(dt)]

    df = pd.DataFrame(flattened_data)

    table = pa.Table.from_pandas(df)
    results.append(table)


In [6]:
import pandas as pd
import pyarrow as pa
import json

def make_new_json(lists,table_name):
    print(table_name)
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            item_data={
                "row_number": i,
                "value": item
            }
            data.append(item_data)
    return data

def delete_lists(df,name):
    
    print("#"*24)
    print("#"*((24-len(name))//2)+name+"#"*((24-len(name))//2))
    print("#"*24)

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()

        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    print(main_table)
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            delete_lists(new_json,dropped_list)

with open(path, encoding='utf-8') as f:
    dt = json.load(f)

delete_lists(dt,'main')



########################
##########main##########
########################
pyarrow.Table
name: string
id: string
nametype: string
recclass: string
mass: string
fall: string
year: string
reclat: string
reclong: string
geolocation.type: string
row_number: int64
----
name: [["Aachen","Aarhus","Abee"]]
id: [["1","2","6"]]
nametype: [["Valid","Valid","Valid"]]
recclass: [["L5","H6","EH4"]]
mass: [["21","720","107000"]]
fall: [["Fell","Fell","Fell"]]
year: [["1880-01-01T00:00:00.000","1951-01-01T00:00:00.000","1952-01-01T00:00:00.000"]]
reclat: [["50.775000","56.183330","54.216670"]]
reclong: [["6.083330","10.233330","-113.000000"]]
geolocation.type: [["Point","Point","Point"]]
...
geolocation.coordinates
########################
geolocation.coordinates
########################
pyarrow.Table
row_number: int64
value: double
----
row_number: [[0,0,1,1,2,2]]
value: [[6.08333,50.775,10.23333,56.18333,-113,54.21667]]


In [7]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                flatten(a, attribute_name)
                i += 1
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out

results = []

for path in paths_test:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)
        
    if isinstance(dt, list):
        flattened_data = [flatten_json(item) for item in dt]
    else:
        flattened_data = [flatten_json(dt)]

    df = pd.DataFrame(flattened_data)

    table = pa.Table.from_pandas(df)
    results.append(table)


In [16]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            item_data={
                "row_number": i,
                "value": item
            }
            data.append(item_data)
    return data

def delete_lists(df,name):

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    tables = [main_table]
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            tables.append(delete_lists(new_json,dropped_list))
    return tables
path=paths_test[3]
with open(path, encoding='utf-8') as f:
    dt = json.load(f)

tables = delete_lists(dt,'main')
print(tables)


[pyarrow.Table
title: string
year: int64
href: string
extract: string
thumbnail: string
thumbnail_width: double
thumbnail_height: double
row_number: int64
----
title: [["After Dark in Central Park","Clowns Spinning Hats","Capture of Boer Battery by British"]]
year: [[1900,1900,1900]]
href: [[null,"Clowns_Spinning_Hats","Capture_of_Boer_Battery_by_British"]]
extract: [[null,"Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900.","Capture of Boer Battery by British is a black-and-white silent short docu-fiction film produced by James H. White for Edison Manufacturing Company in 1900. It is one minute in length and depicts the resistance of the Gordon Highlanders to the oncoming fire of the Boer's advance during the Boer War. It was filmed in West Orange, New Jersey USA and released April 14, 1900."]]
thumbnail: [[null,null,"https://upload.wikimedia.org/wikip

In [34]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            if isinstance(item, dict):
                for key, value in item.items():
                    item_data={
                        "row_number": i,
                        key: value
                    }
                    data.append(item_data)
            else:
                item_data={
                    "row_number": i,
                    "value": item
                }
                data.append(item_data)
    return data

def delete_lists(df,name):

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    tables = [main_table]
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            tables.append(delete_lists(new_json,dropped_list))
    return tables
results=[]
path=paths_test[4]
with open(path, encoding='utf-8') as f:
    dt = json.load(f)

tables = delete_lists(dt,'main')
results.append(tables)


TypeError: 'float' object is not iterable

In [33]:
results[0]

[pyarrow.Table
 name: string
 id: string
 nametype: string
 recclass: string
 mass: string
 fall: string
 year: string
 reclat: string
 reclong: string
 geolocation.type: string
 row_number: int64
 ----
 name: [["Aachen","Aarhus","Abee"]]
 id: [["1","2","6"]]
 nametype: [["Valid","Valid","Valid"]]
 recclass: [["L5","H6","EH4"]]
 mass: [["21","720","107000"]]
 fall: [["Fell","Fell","Fell"]]
 year: [["1880-01-01T00:00:00.000","1951-01-01T00:00:00.000","1952-01-01T00:00:00.000"]]
 reclat: [["50.775000","56.183330","54.216670"]]
 reclong: [["6.083330","10.233330","-113.000000"]]
 geolocation.type: [["Point","Point","Point"]]
 ...,
 [pyarrow.Table
  row_number: int64
  value: double
  ----
  row_number: [[0,0,1,1,2,2]]
  value: [[6.08333,50.775,10.23333,56.18333,-113,54.21667]]]]

In [43]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        if lists[i] is not None and not isinstance(lists[i], float):
            for item in lists[i]:
                if isinstance(item, dict):
                    item_data = item.copy()
                    item_data["row_number"] = i
                else:
                    item_data={
                        "row_number": i,
                        "value": item
                    }
                data.append(item_data)
    return data


def delete_lists(df,name):

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    tables = [main_table]
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            tables += delete_lists(new_json,dropped_list)
    return tables
results=[]
for path in paths:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)

    tables = delete_lists(dt,'main')
    results.append(tables)


In [47]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        if lists[i] is not None and not isinstance(lists[i], float):
            for item in lists[i]:
                if isinstance(item, dict):
                    item_data = item.copy()
                    item_data["row_number"] = i
                else:
                    item_data={
                        "row_number": i,
                        "value": item
                    }
                data.append(item_data)
    return data


def delete_lists(df,name):

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    tables = [main_table]
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            if all(isinstance(item, dict) for item in list_df[dropped_list]):
                flattened_data = [flatten_json(item) for item in list_df[dropped_list]]
            else:
                flattened_data = list_df[dropped_list]
            new_json=make_new_json(flattened_data,dropped_list)
            tables += delete_lists(new_json,dropped_list)
    return tables

results=[]
for path in paths:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)

    tables = delete_lists(dt,'main')
    results.append(tables)


In [48]:
results[3]

[pyarrow.Table
 title: string
 year: int64
 href: string
 extract: string
 thumbnail: string
 thumbnail_width: double
 thumbnail_height: double
 row_number: int64
 ----
 title: [["After Dark in Central Park","Boarding School Girls' Pajama Parade","Buffalo Bill's Wild West Parad","Caught","Clowns Spinning Hats",...,"Aquaman and the Lost Kingdom","Untitled Ghostbusters: Afterlife sequel","Rebel Moon","Migration","The Color Purple"]]
 year: [[1900,1900,1900,1900,1900,...,2023,2023,2023,2023,2023]]
 href: [[null,null,null,null,"Clowns_Spinning_Hats",...,"Aquaman_and_the_Lost_Kingdom","Untitled_Ghostbusters:_Afterlife_sequel","Rebel_Moon","Migration_(2023_film)","The_Color_Purple_(2023_film)"]]
 extract: [[null,null,null,null,"Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900.",...,"Aquaman and the Lost Kingdom is an upcoming American superhero film based on